In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 42.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=1e738a6a031a575f325fa89c6876454877354625eabdcc5d506f31a2c4981040
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

2

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

In [4]:
import re
def parse(line):
    match = re.search("Dm=(\d+)", line)
    if match:
        val = match.group(1)
        return [int(val)]
    return []
    

In [5]:
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc,1)

In [6]:
lines = ssc.socketTextStream("rtd.hpwren.ucsd.edu", 12028)

In [7]:
vals = lines.flatMap(parse)

In [8]:
window = vals.window(10, 5)

In [9]:
def stats(rdd):
    print(rdd.collect())
    if rdd.count() > 0:
        print("max = {}, min = {}".format(rdd.max(), rdd.min()))

In [10]:
window.foreachRDD(lambda rdd: stats(rdd)) 

In [12]:
ssc.start()
ssc.awaitTermination()

max = 295, min = 257
[295, 283, 269, 273, 257, 240, 240, 240, 240, 240]
max = 295, min = 240
[240, 240, 240, 240, 240, 240, 240, 240, 240, 131, 130]
max = 240, min = 130
[240, 240, 240, 240, 131, 130, 130, 130, 131, 131, 131]
max = 240, min = 130
[130, 130, 131, 131, 131, 130, 148, 148, 148, 149]
max = 149, min = 130
[130, 148, 148, 148, 149, 148, 147, 147, 147, 138]
max = 149, min = 130
[148, 147, 147, 147, 138, 138, 139, 134, 134, 134]
max = 148, min = 134
[138, 139, 134, 134, 134, 134, 136, 134, 134, 140]
max = 140, min = 134
[134, 136, 134, 134, 140, 140, 135, 139, 139, 139]
max = 140, min = 134
[140, 135, 139, 139, 139, 139, 139, 140, 136, 138]
max = 140, min = 135
[139, 139, 140, 136, 138, 132, 137, 132, 127, 127]
max = 140, min = 127
[132, 137, 132, 127, 127, 127, 127, 127, 132, 132]
max = 137, min = 127
[127, 127, 127, 132, 132, 138, 138, 144, 148, 148]
max = 148, min = 127
[138, 138, 144, 148, 148, 148, 149, 166, 166, 166]
max = 166, min = 138
[148, 149, 166, 166, 166, 166, 16

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored